In [7]:
import pandas as pd
import numpy as np
from funcs import *
from pandas.tseries.offsets import *
import sys
reload(sys)
sys.setdefaultencoding('utf8')

### 统计季度交易特征

In [8]:
indexs = ['订单总数','订单时间间隔均值（天）','订单时间间隔方差（天）','交易总量','交易量最大值','交易量最小值','交易量方差',
          '交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值','交易稳定性',
          '账期订单占比','账期订单交易量均值','账期订单交易额均值']

In [9]:
def stop_time(start):
    stop = start - DateOffset(months=3, days=start.day)
    return stop

In [10]:
def ac_feature(trade):
    feature = []
    stop = stop_time(trade[u'创建时间'].iloc[0])
    trade_3m = trade[trade[u'创建时间']>stop]
    if trade_3m.shape[0]>0:               
        num = trade_3m.shape[0] #订单数
        feature.append(num)
        tim_st = time_gap(trade_3m[u'创建时间'])
        feature.append(tim_st[0]) #均值
        feature.append(tim_st[1]) #方差
        
        t_num = trade_3m[u'销售数量（吨）'].agg(['sum','max','min','std','mean'])
        t_money = trade_3m[u'销售金额（元）'].agg(['sum','max','min','std','mean'])
        feature.extend(t_num)
        feature.extend(t_money)     
        feature.append(trade_stab_all(trade_3m))
    else:#近三个月无订单
        feature.extend([0]*14) 
        
    zhangqi = trade_3m[trade_3m[u'是否账期']==u'是']
    if zhangqi.shape[0]>0:
        feature.append(zhangqi.shape[0]*1.0/trade_3m.shape[0])
        feature.append(zhangqi[u'销售数量（吨）'].mean())
        feature.append(zhangqi[u'销售金额（元）'].mean())
    else:
        feature.append(0)
        feature.append(0)
        feature.append(0)
    return feature

In [11]:
def df_sub(record,trade):
    df = pd.DataFrame(index=indexs)    
    tmps = zip(record[u'应收单号'],record[u'起算日期'])
    for t in tmps:
        trade_sub = trade[trade[u'创建时间']<t[1]]         
        if trade_sub.shape[0]>1:#有历史订单
            fs = ac_feature(trade_sub.iloc[1:,:])
        else:#新客户
            fs = [np.nan]*len(indexs)
        df[t[0]]=fs
    return df

In [12]:
zq_files = os.listdir('./data/loan_list/')
files = VisitDir('./data/client_data/')
files =[f for f in files if '交易记录' in f]

samples = pd.DataFrame()
for f in zq_files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    record = record.iloc[1:,:]  #账期数据
    for f1 in files:
        if f1.split('/')[-2]==f.split('.')[0]:
            trade = pd.read_excel(f1)   #交易数据
            trade = trade.dropna(thresh=5)
            df = df_sub(record,trade)          
            if samples.shape[0]==0:
                samples=df
            else:
                samples = samples.merge(df,left_index=True,right_index=True,how='left')
            continue

samples =samples.T

In [7]:
samples.to_csv('./new_out/3m_feature_trade.csv')
samples.shape

(1430, 17)

### 季度逾期统计

In [8]:
files = os.listdir('./data/loan_list/')
loan_all = pd.DataFrame()
for f in files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    loan_all = loan_all.append(record.iloc[1:,:],ignore_index=True)
loan_all['zq_day'] = loan_all[u'账期'].apply(lambda x:re.findall('\d+',x)[0])
loan_all['yq_day'] = loan_all[u'逾期'].apply(lambda x:re.findall('^(-\d+|\d+)',x)[0])
loan_all['zq_day'] = loan_all['zq_day'].astype(int)
loan_all['yq_day'] = loan_all['yq_day'].apply(lambda x:int(x))
loan_all=loan_all.drop_duplicates()

yq_ratio =[]
tq_ratio =[]
yq_avg =[]
loan_all[u'起算日期'] =pd.to_datetime(loan_all[u'起算日期'])
for i in xrange(loan_all.shape[0]):
    name = loan_all.iat[i,1]
    time = loan_all.iat[i,10]
    stop = stop_time(time)
    loan_sub = loan_all[(loan_all[u'客户'] == name) & (loan_all[u'起算日期']<time) &(loan_all[u'起算日期']>stop)]
    if loan_sub.shape[0]>0:
        yq_ratio.append(loan_sub[loan_sub['yq_day']>0].shape[0]*1.0 / loan_sub.shape[0])
        tq_ratio.append( loan_sub[loan_sub['yq_day']<0].shape[0]*1.0 / loan_sub.shape[0])
        if loan_sub[loan_sub['yq_day']>0].shape[0]>0:
            yq_avg.append( loan_sub[loan_sub['yq_day']>0]['yq_day'].mean())
        else:
            yq_avg.append(0)
    else:
        yq_ratio.append(np.nan)
        tq_ratio.append(np.nan)
        yq_avg.append(np.nan)

loan_all[u'季度逾期占比']=yq_ratio
loan_all[u'季度提前还款占比']=tq_ratio
loan_all[u'季度平均逾期天数']=yq_avg
loan_all.to_csv('./new_out/3m_loan_all.csv',index=None,encoding='utf-8')
loan_all.head()

,业务类型,客户,已收金额,应收,应收余额,应收单号,应收总额,应收日期,收款状态,账期,起算日期,逾期,销售合同号,zq_day,yq_day,季度逾期占比,季度提前还款占比,季度平均逾期天数
0,账期业务,佛山市南海区松岗明珠塑料有限公司,184520.0,NaN,0.0,YS20170228152309,184520.0,2017-03-30,已收款,30天,2017-02-28,11天,ZSL20170227LL-C003WJ,30,11,0.000000,0.333333,0.0
1,账期业务,佛山市南海区松岗明珠塑料有限公司,171990.0,NaN,0.0,YS20170103150373,171990.0,2017-01-13,已收款,10天,2017-01-03,0天,ZSL20161228LL-C041WJ,10,0,0.000000,0.250000,0.0
2,账期业务,佛山市南海区松岗明珠塑料有限公司,198450.0,NaN,0.0,YS20161117147341,198450.0,2016-12-02,已收款,15天,2016-11-17,-2天,ZSL20161025LL-C009WJ,15,-2,0.125000,0.250000,1.0
3,账期业务,佛山市南海区松岗明珠塑料有限公司,198450.0,NaN,0.0,YS20161115147142,198450.0,2016-11-29,已收款,15天,2016-11-14,0天,ZSL20161025LL-C009WJ,15,0,0.142857,0.285714,1.0
4,账期业务,佛山市南海区松岗明珠塑料有限公司,186940.0,NaN,0.0,YS20161025145675,186940.0,2016-11-09,已收款,15天,2016-10-25,0天,ZSL20161011LL-C038WJ,15,0,0.125000,0.250000,1.0
